In [ ]:
from __future__ import print_function
import os,sys
import ROOT as rt
from larlite import larlite
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np

In [ ]:
f = rt.TFile("icarus_larlite_geodata.root")
t = f.Get("geodata")

In [ ]:
t.GetEntry(0)

In [ ]:
t.cryo_v.at(0)

In [ ]:
def get_tpc_bounds(cryo_v,color=(0,0,0)):
    tpc_plots = []
    for icryo in range(cryo_v.size()):
        cryogeo = cryo_v.at(icryo)
        ntpcs = cryogeo.tpc_v.size()
        for itpc in range(ntpcs):
            #print("tpc[%d]"%(itpc))
            # the four faces
            tpcgeo = cryogeo.tpc_v.at(itpc)
            bounds = np.zeros((3,2))
            for i in range(3):
                bounds[i,0] = tpcgeo.fBounds.at(0)(i)
                bounds[i,1] = tpcgeo.fBounds.at(1)(i)
            #print(bounds)
            top_pts  = [ [bounds[0,0],bounds[1,1], bounds[2,0]],
                          [bounds[0,1],bounds[1,1], bounds[2,0]],
                          [bounds[0,1],bounds[1,1], bounds[2,1]],
                          [bounds[0,0],bounds[1,1], bounds[2,1]],
                          [bounds[0,0],bounds[1,1], bounds[2,0]] ]
            bot_pts  = [ [bounds[0,0],bounds[1,0], bounds[2,0]],
                          [bounds[0,1],bounds[1,0], bounds[2,0]],
                          [bounds[0,1],bounds[1,0], bounds[2,1]],
                          [bounds[0,0],bounds[1,0], bounds[2,1]],
                          [bounds[0,0],bounds[1,0], bounds[2,0]] ]
            up_pts   = [ [bounds[0,0],bounds[1,0], bounds[2,0]],
                          [bounds[0,1],bounds[1,0], bounds[2,0]],
                          [bounds[0,1],bounds[1,1], bounds[2,0]],
                          [bounds[0,0],bounds[1,1], bounds[2,0]],
                          [bounds[0,0],bounds[1,0], bounds[2,0]] ]
            down_pts = [ [bounds[0,0],bounds[1,0], bounds[2,1]],
                          [bounds[0,1],bounds[1,0], bounds[2,1]],
                          [bounds[0,1],bounds[1,1], bounds[2,1]],
                          [bounds[0,0],bounds[1,1], bounds[2,1]],
                          [bounds[0,0],bounds[1,0], bounds[2,1]] ]        
            
            Xe = []
            Ye = []
            Ze = []

            for boundary in [top_pts,bot_pts,up_pts,down_pts]:
                for ipt, pt in enumerate(boundary):
                    Xe.append( pt[0] )
                    Ye.append( pt[1] )
                    Ze.append( pt[2] )
                Xe.append(None)
                Ye.append(None)
                Ze.append(None)
            lines = {
                "type": "scatter3d",
                "x": Xe,
                "y": Ye,
                "z": Ze,
                "mode": "lines",
                "name": "",
                "line": {"color": "rgb(%d,%d,%d)"%color, "width": 5},
            }
            tpc_plots.append(lines)
    return tpc_plots


In [ ]:
detlines = get_tpc_bounds(t.cryo_v)

In [ ]:
def define_circle_mesh( center, radius, value, nsteps=20, color=None, outline_color=None ):

    np_theta = np.linspace(0,2*np.pi,nsteps)
    x = np.ones( nsteps+1 )*center[0]
    y = np.zeros( nsteps+1 )
    z = np.zeros( nsteps+1 )
    y[:nsteps] = np.sin(np_theta)*radius
    z[:nsteps] = np.cos(np_theta)*radius
    y += center[1]
    z += center[2]

    # define vertex indices                                                                                                                                                                   

    i = np.ones( nsteps, dtype=np.int)*nsteps # always points to center                                                                                                                       
    j = np.zeros( nsteps, dtype=np.int )
    k = np.zeros( nsteps, dtype=np.int )
    for n in range(nsteps):
        j[n] = n
        if n+1==nsteps:
            k[n] = 0
        else:
            k[n] = n+1

    if color is None:
        value = min(1.0,value)
        value = max(0.0,value)
        color='rgb(%d,0,0)'%( 54+int(value*200) )

    mesh = go.Mesh3d( x=x, y=y, z=z, i=i, j=j, k=k,
                      showscale=False, name="opflash", color=color )

    lx = np.copy(x)
    ly = np.copy(y)
    lz = np.copy(z)
    lx[nsteps] = lx[0]
    ly[nsteps] = ly[0]
    lz[nsteps] = lz[0]
    if outline_color is None:
        outline_color="rgb(255,255,255)"

    lines = go.Scatter3d( x=lx, y=ly, z=lz, mode="lines", name="opflash", line={"color":outline_color,"width":1} )

    return mesh,lines

def plot_opdets( opdetgeo_v ):
    opdet_traces = []
    for i in range( opdetgeo_v.size() ):
        opdet = opdetgeo_v.at(i)
        mesh,lines = define_circle_mesh( opdet.fCenter, 20, 1.0 )
        opdet_traces.append(mesh)
        opdet_traces.append(lines)
    return opdet_traces

def plot_cryo_opdets( cryo_v ):
    cryo_opdet_traces = []
    for i in range( cryo_v.size() ):
        cryogeo = cryo_v.at(i)
        cryo_opdet_traces += plot_opdets( cryogeo.opdet_v )
    return cryo_opdet_traces

In [ ]:
opdets = plot_cryo_opdets( t.cryo_v )

In [ ]:
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='ICARUS TPC',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 2, "y": 1, "z": 3},
        "camera": {"eye": {"x": -3, "y": 0.1, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=detlines+opdets, layout=layout)
fig.show()
 